In [1]:
import os
from pathlib import Path
import h5py
import pandas as pd
import numpy as np
import arpespythontools as arp
import matplotlib.pyplot as plt
from ipywidgets import widgets, Layout, interactive_output
import astropy 
from mpl_toolkits.mplot3d import Axes3D
import pickle

import os
from pathlib import Path
import h5py
import numpy as np
import pickle

# Input ARPES data here
data_file_path = '20230317_00029.h5'

# Function to generate the cache file path based on the HDF5 file name
def get_cache_file_path(h5_file_path):
    file_name = Path(h5_file_path).stem  # Extract the file name without extension
    return f'{file_name}_cache.pkl'

def load_data(data_file_path):
    # Generate cache file path
    cache_file_path = get_cache_file_path(data_file_path)

    # Check if cache file exists
    if os.path.exists(cache_file_path):
        print(f"Loaded data from cache: {cache_file_path}")
        with open(cache_file_path, 'rb') as f:
            data = pickle.load(f)
        
        # Assign cached data to module-level attributes
        for key, value in data.items():
            globals()[key] = value
        return

    print("Processing data and creating cache...")

    # Load data from HDF5 file
    hf = h5py.File(data_file_path, 'r')

    # Finding map_data name
    map_data_name = str(list(hf['2D_Data'].keys())[0])

    # Reading in map_data
    map_data2 = np.array(hf["2D_Data"][map_data_name][:][:][:])

    # Getting the scaleDelta and offset for energy and theta
    if hf["2D_Data"][map_data_name].attrs["unitNames"][0] == 'eV':
        offset_energy = hf["2D_Data"][map_data_name].attrs["scaleOffset"][0]
        scaleDelta_energy = hf["2D_Data"][map_data_name].attrs["scaleDelta"][0]
        offset_theta = hf["2D_Data"][map_data_name].attrs["scaleOffset"][1]
        scaleDelta_theta = hf["2D_Data"][map_data_name].attrs["scaleDelta"][1]
    elif hf["2D_Data"][map_data_name].attrs["unitNames"][1] == 'eV':
        offset_energy = hf["2D_Data"][map_data_name].attrs["scaleOffset"][1]
        scaleDelta_energy = hf["2D_Data"][map_data_name].attrs["scaleDelta"][1]
        offset_theta = hf["2D_Data"][map_data_name].attrs["scaleOffset"][0]
        scaleDelta_theta = hf["2D_Data"][map_data_name].attrs["scaleDelta"][0]

    # Create axis for phi, energy, and theta
    phi = np.array(hf["0D_Data"]["Slit Defl"][:])
    energy = np.array([offset_energy + scaleDelta_energy * i for i in range(map_data2.shape[0])])
    theta = np.array([offset_theta + scaleDelta_theta * i for i in range(map_data2.shape[1])])

    # Center data for theta
    theta_v = 0.181 * (theta - 533)

    # Save processed data to cache
    data = {'map_data2': map_data2, 'phi': phi, 'energy': energy, 'theta': theta, 'theta_v': theta_v}
    with open(cache_file_path, 'wb') as f:
        pickle.dump(data, f)

    print(f"Cache created: {cache_file_path}")

    # Assign loaded data to module-level attributes
    for key, value in data.items():
        globals()[key] = value

# Load data on import
load_data(data_file_path)


Processing data and creating cache...
Cache created: 20230317_00029_cache.pkl
